In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Steg 1: Definiera Pydantic-modeller för strukturerade utdata

Dessa modeller definierar **schemat** som agenter kommer att returnera. Att använda `response_format` med Pydantic säkerställer:
- ✅ Typ-säker datautvinning
- ✅ Automatisk validering
- ✅ Inga tolkningsfel från fritextsvar
- ✅ Enkel villkorlig routing baserat på fält


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Steg 2: Skapa verktyget för hotellbokning

Det här verktyget är vad **availability_agent** kommer att använda för att kontrollera om rum finns tillgängliga. Vi använder dekorationen `@ai_function` för att:
- Omvandla en Python-funktion till ett AI-anropbart verktyg
- Automatiskt generera JSON-schema för LLM
- Hantera validering av parametrar
- Möjliggöra automatisk anropning av agenter

För denna demo:
- **Stockholm, Seattle, Tokyo, London, Amsterdam** → Har rum ✅
- **Alla andra städer** → Inga rum ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Steg 3: Definiera villkorsfunktioner för routing

Dessa funktioner granskar agentens svar och avgör vilken väg som ska tas i arbetsflödet.

**Viktigt mönster:**
1. Kontrollera om meddelandet är `AgentExecutorResponse`
2. Tolka den strukturerade utmatningen (Pydantic-modell)
3. Returnera `True` eller `False` för att styra routingen

Arbetsflödet kommer att utvärdera dessa villkor på **kanter** för att bestämma vilken exekverare som ska anropas härnäst.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Steg 4: Skapa anpassad visnings-exekutor

Exekutorer är arbetsflödeskomponenter som utför transformationer eller sidoeffekter. Vi använder dekorationen `@executor` för att skapa en anpassad exekutor som visar slutresultatet.

**Viktiga koncept:**
- `@executor(id="...")` - Registrerar en funktion som en arbetsflödesexekutor
- `WorkflowContext[Never, str]` - Typangivelser för in-/utdata
- `ctx.yield_output(...)` - Returnerar det slutliga arbetsflödesresultatet


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Steg 5: Ladda miljövariabler

Konfigurera LLM-klienten. Detta exempel fungerar med:
- **GitHub-modeller** (Gratisnivå med GitHub-token)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Steg 6: Skapa AI-agenter med strukturerade utdata

Vi skapar **tre specialiserade agenter**, var och en innesluten i en `AgentExecutor`:

1. **availability_agent** - Kontrollerar hotellens tillgänglighet med hjälp av verktyget
2. **alternative_agent** - Föreslår alternativa städer (när inga rum finns)
3. **booking_agent** - Uppmuntrar bokning (när rum finns tillgängliga)

**Huvudfunktioner:**
- `tools=[hotel_booking]` - Tillhandahåller verktyget till agenten
- `response_format=PydanticModel` - Tvingar fram strukturerad JSON-utdata
- `AgentExecutor(..., id="...")` - Innesluter agenten för användning i arbetsflödet


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Steg 7: Bygg arbetsflödet med villkorliga kanter

Nu använder vi `WorkflowBuilder` för att konstruera grafen med villkorlig styrning:

**Arbetsflödesstruktur:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Viktiga metoder:**
- `.set_start_executor(...)` - Anger startpunkten
- `.add_edge(from, to, condition=...)` - Lägger till en villkorlig kant
- `.build()` - Slutför arbetsflödet


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Steg 8: Kör testfall 1 - Stad UTAN tillgänglighet (Paris)

Låt oss testa vägen för **ingen tillgänglighet** genom att begära hotell i Paris (som inte har några rum i vår simulering).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Steg 9: Kör testfall 2 - Stad MED tillgänglighet (Stockholm)

Nu ska vi testa **tillgänglighetsvägen** genom att begära hotell i Stockholm (som har rum i vår simulering).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Viktiga insikter och nästa steg

### ✅ Vad du har lärt dig:

1. **WorkflowBuilder-mönster**
   - Använd `.set_start_executor()` för att definiera startpunkt
   - Använd `.add_edge(from, to, condition=...)` för villkorlig routing
   - Anropa `.build()` för att slutföra arbetsflödet

2. **Villkorlig routing**
   - Villkorsfunktioner inspekterar `AgentExecutorResponse`
   - Tolka strukturerade utdata för att fatta routingbeslut
   - Returnera `True` för att aktivera en kant, `False` för att hoppa över den

3. **Verktygsintegration**
   - Använd `@ai_function` för att konvertera Python-funktioner till AI-verktyg
   - Agenter anropar verktyg automatiskt vid behov
   - Verktyg returnerar JSON som agenter kan tolka

4. **Strukturerade utdata**
   - Använd Pydantic-modeller för typ-säker datautvinning
   - Ställ in `response_format=MyModel` när du skapar agenter
   - Tolka svar med `Model.model_validate_json()`

5. **Anpassade exekutorer**
   - Använd `@executor(id="...")` för att skapa arbetsflödeskomponenter
   - Exekutorer kan transformera data eller utföra sidoeffekter
   - Använd `ctx.yield_output()` för att producera arbetsflödesresultat

### 🚀 Verkliga tillämpningar:

- **Resebokning**: Kontrollera tillgänglighet, föreslå alternativ, jämför val
- **Kundservice**: Routen baserat på ärendetyp, känslor, prioritet
- **E-handel**: Kontrollera lager, föreslå alternativ, behandla beställningar
- **Innehållsmoderering**: Routen baserat på toxicitetspoäng, användarflaggor
- **Godkännandeflöden**: Routen baserat på belopp, användarroll, risknivå
- **Bearbetning i flera steg**: Routen baserat på datakvalitet, fullständighet

### 📚 Nästa steg:

- Lägg till mer komplexa villkor (flera kriterier)
- Implementera loopar med arbetsflödeshantering
- Lägg till sub-arbetsflöden för återanvändbara komponenter
- Integrera med riktiga API:er (hotellbokning, lagersystem)
- Lägg till felhantering och reservvägar
- Visualisera arbetsflöden med de inbyggda visualiseringsverktygen



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör det noteras att automatiserade översättningar kan innehålla fel eller felaktigheter. Det ursprungliga dokumentet på dess originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
